In [1]:
from vector_pipeline import connect_db, table_exists,create_table_if_not_exists,load_csv_to_db
from retriever import format_results, get_common_results, rerag
from langchain.prompts import PromptTemplate
from langchain_mistralai import ChatMistralAI
from dotenv import load_dotenv
import os


c:\Users\kingr\anaconda3\lib\site-packages\langchain_mistralai\embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [2]:
api_key = os.getenv('MISTRAL_API')




connection, cursor = connect_db()


In [3]:
query = "what are the scholarships for indian students?"

In [4]:
results = get_common_results(query, 'edu', cursor=cursor, top_k=3)

In [5]:
results

[(130,
  'https://ernest.essec.edu/support/solutions/articles/7000035125--full-time-student-i-m-from-india-is-there-a-scholarship-i-can-apply-for-',
  'Financial Aid : scholarships, sponsorship / bourses, prise en charge',
  'Financial Aid : scholarships, sponsorship / bourses, prise en charge\nCharpak scholarship\n\n  * [https://www.inde.campusfrance.org/](https://www.inde.campusfrance.org/)\n  * Deadline for the [Charpak](https://www.inde.campusfrance.org/charpak-master-s-scholarship) excellence scholarship grant : deadline 20 March 2024\n\n** _  \n_**\n\n[https://www.inde.campusfrance.org/charpark-master-s-\nprogram](https://www.inde.campusfrance.org/charpark-master-s-program)).\n\nLegrand scholarship\n\n[https://www.inde.campusfrance.org/ugam-legrand-scholarship-\nprogram](https://www.inde.campusfrance.org/ugam-legrand-scholarship-program)\n\n  \n\n  \n\n  \n\n  \n\n',
  'Mar, 30 Janv., 2024 à  4:55 H',
  None),
 (196,
  'https://ernest.essec.edu/support/solutions/articles/70000570

In [6]:
results, f_ques, e_ques= rerag(query, 'edu', cursor=cursor, top_k=3)

context = format_results(results)

In [7]:
f_ques

"Quelles sont les bourses d'études pour les étudiants indiens ?"

In [8]:
e_ques

'What are the scholarships for Indian students?'

In [9]:
results

[(130,
  'https://ernest.essec.edu/support/solutions/articles/7000035125--full-time-student-i-m-from-india-is-there-a-scholarship-i-can-apply-for-',
  'Financial Aid : scholarships, sponsorship / bourses, prise en charge',
  'Financial Aid : scholarships, sponsorship / bourses, prise en charge\nCharpak scholarship\n\n  * [https://www.inde.campusfrance.org/](https://www.inde.campusfrance.org/)\n  * Deadline for the [Charpak](https://www.inde.campusfrance.org/charpak-master-s-scholarship) excellence scholarship grant : deadline 20 March 2024\n\n** _  \n_**\n\n[https://www.inde.campusfrance.org/charpark-master-s-\nprogram](https://www.inde.campusfrance.org/charpark-master-s-program)).\n\nLegrand scholarship\n\n[https://www.inde.campusfrance.org/ugam-legrand-scholarship-\nprogram](https://www.inde.campusfrance.org/ugam-legrand-scholarship-program)\n\n  \n\n  \n\n  \n\n  \n\n',
  'Mar, 30 Janv., 2024 à  4:55 H',
  None),
 (196,
  'https://ernest.essec.edu/support/solutions/articles/70000570

In [10]:
from prompts import system_prompt_template, custom_answer_prompt_template

In [11]:
answer_prompt = PromptTemplate(template=custom_answer_prompt_template, input_variables=['context', 'query'])

formatted_prompt = answer_prompt.format(query=query, context=context)

llm = ChatMistralAI(model_name='mistral-large-latest',api_key='jEoYKB36Y0ClSdn2n1cuLziCuyqntQzf', streaming= True, system = system_prompt_template)





In [13]:
answer = llm.invoke(formatted_prompt)

In [14]:
print(answer.content)

There are a few scholarships available for Indian students as per the provided documents:

1. Charpak Scholarship: This scholarship is for Indian students who wish to pursue their Master's degree in France. The deadline for the Charpak excellence scholarship grant is 20th March 2024. For more information, visit [Charpak Master's Scholarship](https://www.inde.campusfrance.org/charpak-master-s-scholarship) or write to scholarship.france@ifindia.in.

2. Charpak Exchange Scholarship: This scholarship is open to students from all streams and fields of study enrolled in an Indian institution at Bachelor’s and Master’s degree level. It is offered for one academic semester (4 months) between September and December. Applications for exchange in the Fall must be submitted between January and April, and for the Spring between September and November. For more information, visit [Charpak Exchange Scholarship](https://www.inde.campusfrance.org/charpak-exchange-scholarship-autumn-session-sept-dec-0) 

In [21]:
from langchain.agents import AgentExecutor, initialize_agent
from langchain.tools import BaseTool
import re


# Define the function to raise a ticket
def raise_ticket(query):
    # Logic to raise a ticket with the user's query
    ticket_id = 1234
    return f"Ticket raised successfully with ID: {ticket_id}"

# Create a custom tool
class RaiseTicketTool(BaseTool):
    name = "raise_ticket_tool"
    description = "Prompts the user to raise a ticket if the LLM cannot answer the query."

    def _run(self, query: str) -> str:
        # Prompt the user
        prompt = f"I'm sorry, I couldn't find a good answer to your query: '{query}'. Do you want to raise a ticket? (Yes/No)"
        user_response = input(prompt).lower()

        # Check if the user wants to raise a ticket
        if re.search(r'yes', user_response):
            return raise_ticket(query)
        else:
            return "Okay, no ticket will be raised."

    def _arun(self, query: str) -> str:
        raise NotImplementedError("This tool does not support async")



# Initialize the agent with the custom tool and output key
tools = [RaiseTicketTool()]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True, output_keys=["result"])

# Create the AgentExecutor
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

# Use the agent
response = agent_executor.run("What is the capital of France?")
print(response["result"])

ValueError: `run` not supported when there is not exactly one output key. Got [].